# QFin Trading Team Workshop 3
## Creating Custom Indicators

### Recap of Last Week

In [1]:
import pandas as pd

#### Read In APPL Data

In [2]:
aapl_df = pd.read_csv('./data/AAPL.csv', index_col='time')
aapl_df.head()

,open,high,low,close,volume
time,,,,,
2021-04-09 09:31:00,128.095547,128.480426,128.095547,128.312659,1728495.0
2021-04-09 09:32:00,128.302790,128.391608,128.031401,128.154760,595045.0
2021-04-09 09:33:00,128.149825,128.233709,127.908042,127.977123,567205.0
2021-04-09 09:34:00,127.977222,128.095547,127.878436,128.006729,495596.0
2021-04-09 09:35:00,128.001795,128.085679,127.868568,127.908042,399394.0


#### Create Custom Indicator

In [3]:
aapl_df['mid_price'] = (aapl_df['high'] + aapl_df['low']) / 2
aapl_df = aapl_df.drop(columns=['open', 'high', 'low', 'close'])

BOLLINGER_WIDTH = 2
rolling_mid = aapl_df['mid_price'].rolling(100).mean()
rolling_std = aapl_df['mid_price'].rolling(100).std()
aapl_df['upper_bollinger'] = rolling_mid + BOLLINGER_WIDTH*rolling_std
aapl_df['lower_bollinger'] = rolling_mid - BOLLINGER_WIDTH*rolling_std

aapl_df.dropna(inplace=True)
aapl_df

,volume,mid_price,upper_bollinger,lower_bollinger
time,,,,
2021-04-09 11:10:00,189778.0,128.949189,129.969133,128.127874
2021-04-09 11:11:00,216297.0,128.978795,129.963262,128.147561
2021-04-09 11:12:00,132587.0,128.949189,129.954783,128.170794
2021-04-09 11:13:00,157577.0,128.934336,129.941059,128.201787
2021-04-09 11:14:00,184305.0,129.028138,129.923493,128.240176
...,...,...,...,...
2022-12-29 15:56:00,368260.0,129.630063,129.998280,129.515498
2022-12-29 15:57:00,282248.0,129.595117,129.996215,129.511822
2022-12-29 15:58:00,254365.0,129.557624,129.994414,129.506942


In [4]:
%pip install qfinuwa

Note: you may need to restart the kernel to use updated packages.


### Pulling Data with the API

In [5]:
from qfinuwa import API

In [6]:
# API.fetch_stocks(['AAPL', 'TSLA', 'GOOG'], 'API_key.txt', './data')

### Implement with QFin Backtester

In [7]:
from qfinuwa import Indicators

In [8]:

# Extend base indicator class

class CustomIndicators(Indicators):
    
    ...





The backtester expects indicators to be a function that takes in data (``pd.DataFrame(s)``) and return indicators. 

There are two types of indicators in the backtester - SingleIndicators and MultiIndicators.

### Multi-Indicators

A multi-indicator takes in a single signal (price of an arbitary stock) and outputs a transformation of that stock.

```py
f(stock) -> (indicator on stock)
``` 

It is called ``MultiIndicator`` because the indicator will have multiple values (one for each stock)

In [9]:
import pandas as pd
class CustomIndicators(Indicators):
    
    @Indicators.MultiIndicator
    def bollinger_bands(self, stock: pd.DataFrame):
        BOLLINGER_WIDTH = 2
        WINDOW_SIZE = 100
        
        mid_price = (stock['high'] + stock['low']) / 2
        rolling_mid = mid_price.rolling(WINDOW_SIZE).mean()
        rolling_std = mid_price.rolling(WINDOW_SIZE).std()

        return {"upper_bollinger": rolling_mid + BOLLINGER_WIDTH*rolling_std,
                "lower_bollinger": rolling_mid - BOLLINGER_WIDTH*rolling_std}

### Testing Multi Indicator

In [10]:

# indicators = CustomIndicators(data='./data')

# indicator_values = indicators.indicator_values()

# indicator_values["upper_bollinger"], type(indicator_values["upper_bollinger"])
# indicator_values["lower_bollinger"], type(indicator_values["lower_bollinger"])

In [11]:
# indicator_values["upper_bollinger"]['AAPL']

### Plotting

In [12]:
# import matplotlib.pyplot as plt

# plt.plot(indicators.index,  indicator_values["upper_bollinger"]['AAPL'])
# plt.plot(indicators.index,  indicator_values["lower_bollinger"]['AAPL'])

### SingleIndicator

Similar to ``MultiIndicator``, ``SingleIndicator`` is implemented as a function that takes in stock data and returns an indicator or indicators.

```py
f(list_of_stocks) -> (single indicator on stocks)
``` 

It is called ``SingleIndicator`` because there is only a single signal.

In [13]:
class CustomIndicators(Indicators):
    
    @Indicators.MultiIndicator
    def bollinger_bands(self, stock: pd.DataFrame):
        BOLLINGER_WIDTH = 2
        WINDOW_SIZE = 100
        
        mid_price = (stock['high'] + stock['low']) / 2
        rolling_mid = mid_price.rolling(WINDOW_SIZE).mean()
        rolling_std = mid_price.rolling(WINDOW_SIZE).std()

        return {"upper_bollinger": rolling_mid + BOLLINGER_WIDTH*rolling_std,
                "lower_bollinger": rolling_mid - BOLLINGER_WIDTH*rolling_std}
    
    @Indicators.SingleIndicator
    def etf(self, stock: dict):

        apple = 0.2
        tsla = 0.5
        goog = 0.3

        return {'etf': apple*stock['AAPL'] + tsla*stock['TSLA'] + goog*stock['GOOG']}


In [14]:
indicators = CustomIndicators(data='./data')

indicator_values = indicators.indicator_values()
indicator_values.keys()

TypeError: 'NoneType' object is not iterable

In [ ]:

print(type(indicator_values["etf"]))

print(type(indicator_values["upper_bollinger"]))
print(type(indicator_values["upper_bollinger"]['AAPL']))

### Hyperparameters

Notice in each indicator we have some constant values that are arbitarily chosen. We might choose to optimise those parameters on our training data.


The Backtester can do this for you, if you format them with ``kwargs``.

Each ``function`` you implemented acts as a "indicator parameter group", where the paramters attached to it affect the indicators returned by it.

In [ ]:
class CustomIndicators(Indicators):
    
    @Indicators.MultiIndicator
    def bollinger_bands(self, stock: pd.DataFrame, BOLLINGER_WIDTH = 2, WINDOW_SIZE = 100):

        mid_price = (stock['high'] + stock['low']) / 2
        rolling_mid = mid_price.rolling(WINDOW_SIZE).mean()
        rolling_std = mid_price.rolling(WINDOW_SIZE).std()

        return {"upper_bollinger": rolling_mid + BOLLINGER_WIDTH*rolling_std,
                "lower_bollinger": rolling_mid - BOLLINGER_WIDTH*rolling_std}
    
    @Indicators.SingleIndicator
    def etf(self, stock: dict, apple = 0.2, tsla = 0.5, goog = 0.3):

        return {'etf':  apple*stock['AAPL']['close'] + \
                        tsla*stock['TSLA']['close'] +\
                        goog*stock['GOOG']['close']}

In [ ]:
## SIDENOTE: KEYWORD ARGS

def foo(a, b, c):
    print(a, b, c)

foo(1, 2, 3)

def foo_kwarg(a, b=2, c=3):
    print(a, b, c)

foo_kwarg(1)

In [ ]:
indicators = CustomIndicators(stockdata='./data')

indicators.defaults

In [ ]:
indicators.indicator_values()['etf']

In [ ]:
indicators.update_parameters({'etf': {'apple': 0.5, 'tsla': 0.2, 'goog': 0.3}})

indicators.indicator_values()['etf']

### Now Make Your Own!

In [ ]:
class MyIndicator(Indicators):
    ...